# Dynamic

Some of the information in dynamic statistics is security restricted - ordinary users can only

## What To Monitor

- Processes and their commands
- Replication
- Connections
- Backend operations progress (`ANALYZE`, `CLUSTER`, `CREATE INDEX`, `VACUUM`, `COPY`, `BASE_BACKUP`)

## How To Monitor

Category    |    View Name                      |   Description
--------    |    -------                        |   --------
Processes   |   pg_stat_activity	            |   One row per server process, showing information related to the current activity of that process, such as state and current query
Replication |   pg_stat_replication	            |   One row per WAL sender process, showing statistics about replication to that sender's connected standby server
Replication |   pg_stat_wal_receiver	        |   Only one row, showing statistics about the WAL receiver from that receiver's connected server
Replication |   pg_stat_recovery_prefetch	    |   Only one row, showing statistics about blocks prefetched during recovery
Replication |   pg_stat_subscription	        |   At least one row per subscription, showing information about the subscription workers
Connections |   pg_stat_ssl	                    |   One row per connection (regular and replication), showing information about SSL used on this connection
Connections |   pg_stat_gssapi	                |   One row per connection (regular and replication), showing information about GSSAPI authentication and encryption used on this connection
Progress    |   pg_stat_progress_analyze	    |   One row for each backend (including autovacuum worker processes) running ANALYZE, showing current progress
Progress    |   pg_stat_progress_create_index   |	One row for each backend running CREATE INDEX or REINDEX, showing current progress
Progress    |   pg_stat_progress_vacuum         |	One row for each backend (including autovacuum worker processes) running VACUUM, showing current progress
Progress    |   pg_stat_progress_cluster        |	One row for each backend running CLUSTER or VACUUM FULL, showing current progress
Progress    |   pg_stat_progress_basebackup     |	One row for each WAL sender process streaming a base backup, showing current progress
Progress    |   pg_stat_progress_copy           |	One row for each backend running COPY, showing current progress

More information in the [docs](https://www.postgresql.org/docs/current/monitoring-stats.html#MONITORING-STATS-DYNAMIC-VIEWS-TABLE)

## Usage Example

### Long Running Queries

In [ ]:
SELECT (NOW() - xact_start) running_time, *
FROM pg_stat_activity
WHERE state IN ('idle in transaction', 'active');

### Connections By State

In [ ]:
SELECT state, count(*) num
FROM (
    SELECT 
        CASE 
            WHEN state IS NULL THEN 'system_process' 
            ELSE state 
        END 
    FROM pg_stat_activity
)
GROUP BY state;

### Blocked Queries

In [ ]:
SELECT
    pid,
    usename,
    pg_blocking_pids(pid) AS blocked_by,
    query AS blocked_query
FROM pg_stat_activity
WHERE cardinality(pg_blocking_pids(pid)) > 0;